In [1]:
import pandas
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from MyCode import relabeling_fairlearn
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

In [2]:
round_value = 15

In [3]:
if True:
    from tqdm import tqdm
    from fairlearn.datasets import fetch_adult

    dataset = fetch_adult(as_frame=True)
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    df['>50k'] = (dataset.target == '>50K') * 1


    le = preprocessing.LabelEncoder()
    df['sex'] = le.fit_transform(df['sex'])

    onehot = ['workclass', 'education', 'marital-status', 'occupation', 'marital-status', 'occupation', 'relationship', 'race']
    df = pd.get_dummies(df,prefix=onehot, columns = onehot, drop_first=True)
    tmp = df['sex'].to_list()
    for i in range(0,len(tmp)):
        if tmp[i] == 1:
            tmp[i] = 0
        else:
            tmp[i] = 1
    df['sex'] = tmp
    y = df[">50k"]
    sensitive = df['sex']
    X = df.loc[:, ~df.columns.isin(['sex', '>50k', 'native-country'])]
    print(relabeling_fairlearn.discrimination_dataset(y, sensitive))


0.19451574596420296


In [4]:
if False:
    pd.set_option('max_columns', None)
    FILE_NAME = "relab_Lawsuit"

    # src: https://www.kaggle.com/hjmjerry/gender-discrimination
    df = pandas.read_csv("dataset_perso/Lawsuit.csv")
    del df['ID']
    salary_mean = np.mean(np.array(df["Sal94"].tolist(), float))
    for i in range(0,len(df)):
        if df.at[i, "Sal94"] >= salary_mean:
            df.at[i, "Salary_mean"] = 1
        else:
            df.at[i, "Salary_mean"] = 0
    del df["Sal94"]
    del df["Sal95"]
    col = ['Rank', 'Dept']
    df = df.drop(['Prate', 'Exper'], axis=1)
    df = pd.get_dummies(df, columns=col)
    for i in range(0,len(df)):
        if df.at[i, "Gender"] == 1:
            df.at[i, "Gender"] = 0
        else:
            df.at[i, "Gender"] = 1
    X = df.loc[:, ~df.columns.isin(['Gender', 'Salary_mean'])]
    y = df['Salary_mean']
    sensitive = df['Gender']

In [5]:
if False:
    pd.set_option('max_columns', None)
    FILE_NAME = "HR"

    # src: https://www.kaggle.com/hjmjerry/gender-discrimination
    df = pandas.read_csv("dataset_perso/HRDataset_v14.csv")
    del df['Employee_Name']
    del df['EmpID']
    salary_mean = np.mean(np.array(df["Salary"].tolist(), float))
    print(f"Mean: {salary_mean}")
    for i in range(0,len(df)):
        if df.at[i, "Salary"] >= salary_mean:
            df.at[i, "Salary_mean"] = 1
        else:
            df.at[i, "Salary_mean"] = 0
    del df["Salary"]

    for i in range(0,len(df)):
        df.at[i, "Absences"] = df.at[i, "Absences"] / 5
    for i in range(0,len(df)):
        df.at[i, "EngagementSurvey"] = int(df.at[i, "EngagementSurvey"])
    for i in range(0,len(df)):
        if df.at[i, "HispanicLatino"] == 'Yes' or df.at[i, "HispanicLatino"] == 'yes' :
            df.at[i, "HispanicLatino"] = 1
        elif df.at[i, "HispanicLatino"] == 'No' or df.at[i, "HispanicLatino"] == 'no' :
            df.at[i, "HispanicLatino"] = 0
    col = ['EmpStatusID', 'PerfScoreID', 'Position', 'MaritalDesc', 'CitizenDesc', 'RaceDesc', 'Department', 'PerformanceScore', 'EmpSatisfaction', 'Absences']
    df = df.drop(['MarriedID', 'MaritalStatusID', 'Zip', 'DOB', 'Sex', 'DateofHire','DateofTermination', 'TermReason', 'EmploymentStatus', 'ManagerName', 'ManagerID', 'EngagementSurvey', 'LastPerformanceReview_Date', 'DaysLateLast30', 'RecruitmentSource', 'State', 'DeptID', 'PositionID', 'SpecialProjectsCount'], axis=1)
    df = pd.get_dummies(df, columns=col)
    for col in df:
        if len(df[col].unique()) > 2:
            print(f'{col}: {df[col].unique()}')
    for i in range(0,len(df)):
        if df.at[i, "GenderID"] == 1:
            df.at[i, "GenderID"] = 0
        else:
            df.at[i, "GenderID"] = 1
    X = df.loc[:, ~df.columns.isin(['Gender', 'Salary_mean'])]
    y = df['Salary_mean']
    sensitive = df['GenderID']
    print(relabeling_fairlearn.discrimination_dataset(y, sensitive))

In [6]:
if False:
    data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
    data_url = "dataset_perso/default of credit card clients.xls"
    dataset = pd.read_excel(io=data_url, header=1).drop(columns=['ID']).rename(columns={'PAY_0':'PAY_1'})

    tmp = dataset['SEX'].to_list()
    for i in range(0,len(tmp)):
        if tmp[i] == 1:
            tmp[i] = 0
        elif tmp[i] == 2:
            tmp[i] = 1
        else:
            raise Exception(i)

    dataset["SEX"] = tmp
    sensitive = dataset["SEX"]
    y = dataset["default payment next month"]
    X = dataset.loc[:, ~dataset.columns.isin(['SEX', 'default payment next month'])]
    categorical_features = ['EDUCATION', 'MARRIAGE','PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
    for col in categorical_features:
        dataset[col] = dataset[col].astype('category')

In [7]:
def train_test_split(random_state, X, y, sensitive):
    index_train = list(X.sample(frac=0.8, random_state=random_state).index)
    index_test = list(X.drop(index=index_train).index)

    X_train = X.drop(index=index_test).to_numpy()
    y_train = y.drop(index=index_test).to_numpy()
    sensitive_train = sensitive.drop(index=index_test).to_numpy()

    X_test = X.drop(index=index_train).to_numpy()
    y_test = list(y.drop(index=index_train).to_numpy())
    sensitive_test = sensitive.drop(index=index_train).to_numpy()

    return X_train, X_test, y_train, y_test, sensitive_train, sensitive_test

X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(1, X, y, sensitive)
print(relabeling_fairlearn.discrimination_dataset(y_train, sensitive_train))

0.1925655508416967


# Basic usage

In [8]:
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, list(y_train))
y_pred = clf.predict(X_train)
accuracy = round(accuracy_score(y_train, y_pred),round_value)
print("Accuracy of the prediction: ", accuracy)
discrimination= round(relabeling_fairlearn.discrimination_dataset(y_pred, sensitive_train), round_value)
print("Discrimination of classifier on the predidiction: ", discrimination)

Accuracy of the prediction:  0.839612018221836
Discrimination of classifier on the predidiction:  0.18826592982247


In [9]:
"""
# Raises an exception
sensitive = list(sensitive)
sensitive[0] = 2
sensitive = np.asarray(sensitive)
print(sensitive)
print(np.unique(sensitive))
"""
threshold = 0.1

In [10]:
relabeling_fairlearn.relabeling(clf, X_train, y_train, y_pred, sensitive_train, threshold)
y_pred_relab = clf.predict(X_train)
accuracy_relab = round(accuracy_score(y_train, y_pred_relab),round_value)
print("Accuracy of the prediction: ", accuracy_relab)
discrimination_relab = round(relabeling_fairlearn.discrimination_dataset(y_pred_relab, sensitive_train), round_value)
print("Discrimination of classifier on the prediction: ", discrimination_relab)

Unable to reach the threshold.
Accuracy of the prediction:  0.768311409121155
Discrimination of classifier on the prediction:  0.055832129864993


# Detailed operations

In [11]:
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, list(y_train))
y_pred = clf.predict(X_train)
accuracy = round(accuracy_score(y_train, y_pred), round_value)
print("Accuracy of the prediction: ", accuracy)
discrimination= round(relabeling_fairlearn.discrimination_dataset(y_pred, sensitive_train), round_value)
print("Discrimination of classifier on the prediction: ", discrimination)

Accuracy of the prediction:  0.839612018221836
Discrimination of classifier on the prediction:  0.18826592982247


In [12]:
threshold = 0

In [13]:
i = list()
cnt = np.unique(sensitive, return_counts=True)[1]
relabeling_fairlearn.get_leaves_candidates(clf, X_train, y_train, sensitive_train, cnt, len(y), i)
for e in i:
    print(e)

Path: format -> (feature, type, node id)
((67, 'left', 0), (3, 'left', 1), (2, 'right', 2), (-2, 'leaf', 4)) 
node_id: 4 
The effect of relabeling the leaf on accuracy: -0.0009213381925392081
The effect of relabeling the leaf on discrimination: -0.000284707552251996 
-(18 + 0) / 16192 + (27 + 0) / 32650= -0.000284707552251996 
ratio: 0.30901525037982197 
contingency table: 
[0.0003685352770156832, 0.0]
[0.0005528029155235249, 0.0]
transactions: [838, 861, 1205, 1924, 2002, 3549, 5871, 6198, 6227, 8590, 8634, 8777, 9169, 9354, 10860, 11342, 12542, 13207, 15930, 18716, 19385, 19534, 19538, 19545, 20077, 21294, 23631, 25090, 26302, 27750, 30138, 30150, 30474, 30764, 31370, 33072, 33081, 33171, 33510, 33735, 34696, 34762, 35707, 38912, 38972]


In [14]:
leafs_relab = relabeling_fairlearn.leaves_to_relabel(clf, X_train, y_train, y_pred, sensitive_train, threshold)

sum_acc = 0
sum_disc = 0
for l in leafs_relab:
    print(l)
    print()
    sum_acc += l.acc
    sum_disc += l.disc
sum_acc = round(sum_acc, round_value)
sum_disc = round(sum_disc, round_value)

Unable to reach the threshold.
Path: format -> (feature, type, node id)
((67, 'left', 0), (3, 'left', 1), (2, 'right', 2), (-2, 'leaf', 4)) 
node_id: 4 
The effect of relabeling the leaf on accuracy: -0.0011516609510160208
The effect of relabeling the leaf on discrimination: -0.00036334813001866325 
-(18 + 0) / 12903 + (27 + 0) / 26171= -0.00036334813001866325 
ratio: 0.31549921849664997 
contingency table: 
[0.00046066438040640837, 0.0]
[0.0006909965706096125, 0.0]
transactions: [838, 861, 1205, 1924, 2002, 3549, 5871, 6198, 6227, 8590, 8634, 8777, 9169, 9354, 10860, 11342, 12542, 13207, 15930, 18716, 19385, 19534, 19538, 19545, 20077, 21294, 23631, 25090, 26302, 27750, 30138, 30150, 30474, 30764, 31370, 33072, 33081, 33171, 33510, 33735, 34696, 34762, 35707, 38912, 38972]



In [15]:
relabeling_fairlearn.relabeling(clf, X_train, y_train, y_pred, sensitive_train, threshold)
y_pred_relab = clf.predict(X_train)
accuracy_relab = round(accuracy_score(y_train, y_pred_relab), round_value)
discrimination_relab = round(relabeling_fairlearn.discrimination_dataset(y_pred_relab, sensitive_train), round_value)
new_acc= round(accuracy+sum_acc, round_value)
new_disc = round(discrimination+sum_disc, round_value)
print(f"Accuracy:\n"
    f"    Before      : {accuracy}\n"
    f"    After       : {accuracy_relab}\n"
    f"    Expected    : {new_acc}\n"
    f"    Leafs       : {sum_acc}\n"
    f"    Difference  : {-accuracy+accuracy_relab}\n"
    f"    Check       : {new_acc == accuracy_relab}")
print(f"Discrimination:\n"
    f"    Before      : {discrimination}\n"
    f"    After       : {discrimination_relab}\n"
    f"    Expected    : {new_disc}\n"
    f"    Leafs       : {sum_disc }\n"
    f"    Difference  : {-discrimination+discrimination_relab}\n"
    f"    Check       : {new_disc== discrimination_relab}")

Unable to reach the threshold.
Accuracy:
    Before      : 0.839612018221836
    After       : 0.768311409121155
    Expected    : 0.83846035727082
    Leafs       : -0.001151660951016
    Difference  : -0.07130060910068092
    Check       : False
Discrimination:
    Before      : 0.18826592982247
    After       : 0.055832129864993
    Expected    : 0.187902581692451
    Leafs       : -0.000363348130019
    Difference  : -0.132433799957477
    Check       : False


In [16]:
l = list(leafs_relab)[0]
print(l)

Path: format -> (feature, type, node id)
((67, 'left', 0), (3, 'left', 1), (2, 'right', 2), (-2, 'leaf', 4)) 
node_id: 4 
The effect of relabeling the leaf on accuracy: -0.0011516609510160208
The effect of relabeling the leaf on discrimination: -0.00036334813001866325 
-(18 + 0) / 12903 + (27 + 0) / 26171= -0.00036334813001866325 
ratio: 0.31549921849664997 
contingency table: 
[0.00046066438040640837, 0.0]
[0.0006909965706096125, 0.0]
transactions: [838, 861, 1205, 1924, 2002, 3549, 5871, 6198, 6227, 8590, 8634, 8777, 9169, 9354, 10860, 11342, 12542, 13207, 15930, 18716, 19385, 19534, 19538, 19545, 20077, 21294, 23631, 25090, 26302, 27750, 30138, 30150, 30474, 30764, 31370, 33072, 33081, 33171, 33510, 33735, 34696, 34762, 35707, 38912, 38972]


In [17]:
import utils
utils.discr_add(l.transactions, y_train, sensitive_train)

(0, 0) (26171, 12903)
(0 / 26171) - (0 / 12903))


0.0